In [2]:
import wfdb
import numpy as np
import os

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,  Dense, Dropout, GlobalMaxPooling1D,BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from keras.regularizers import l2


import pandas as pd
import matplotlib.pyplot as plt

import time

from imblearn.over_sampling import SMOTE

In [ ]:
RECORD_SIZE=262124  # 262124 Samples 
RECORD_TIME=11.18   #11.18 Second
SAMLING_RATE=0
WIN_SIZE = 23437 # 1 Second window
OVERLAP = 14062 # 0.6 Second

#CLASSES={C:Control A:ALS  M:Myopathy}

In [3]:
def getStat(muscle,disease, label_map):
    k=0
    for i in range (len(label_map)):
        if muscle in label_map[i]:
            if disease in label_map[i]:
                k=k+1
    return k

In [4]:
def rescale_data(X_train, X_test, X_val):
    scaler = StandardScaler()     #subtracts mean and divides by std 

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_val = scaler.transform(X_val)
    return X_train ,X_test ,X_val

In [5]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
        arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
        arg = tf.reshape(arg,dim)
        dataset.append(arg)

    return dataset

In [6]:
def windowing_fun(data,window_size=WIN_SIZE,overlap=OVERLAP):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data

In [7]:
def remove_nan(arr,Y):
    # Find the rows with NaN values
    nan_rows = np.isnan(arr).any(axis=1)

    # Convert the boolean array to integer indices
    index_to_remove = np.where(nan_rows)[0]

    # Remove the rows with NaN values
    arr = np.delete(arr, index_to_remove, axis=0)
    Y = np.delete(Y, index_to_remove, axis=0)

    return arr,Y

In [9]:
def preprocessing(X_train,X_val,X_test,Y_train,Y_val,Y_test):

    #  Windowing
    X_train = windowing_fun(X_train, WIN_SIZE)
    X_val = windowing_fun(X_val, WIN_SIZE)
    X_test = windowing_fun(X_test, WIN_SIZE)

   # NaN removal
    X_train,Y_train = remove_nan(X_train,Y_train)
    X_val,Y_val = remove_nan(X_val,Y_val)
    X_test,Y_test = remove_nan(X_test,Y_test)
    
    # Normalization
    X_train, X_test, X_val = rescale_data(X_train,X_test,X_val)
    
    # Convert to tensor

    #11719 ???
    tensors_list_train = gen_tensors_list(X_train,(11719,1))
    tensors_list_val = gen_tensors_list(X_val,(11719,1))
    tensors_list_test = gen_tensors_list(X_test,(11719,1))
    
    return tensors_list_train,tensors_list_val,tensors_list_test,Y_train,Y_val,Y_test

In [10]:
def getClassesStat(Y, X):
    last_index_als = len(Y) - 1 - Y[::-1].index(0) if 0 in Y else -1
    last_index_normal = len(Y) - 1 - Y[::-1].index(1) if 1 in Y else -1
    last_index_myopathie = len(Y) - 1 - Y[::-1].index(2) if 2 in Y else -1
    
    windowed_als = len(windowing_fun(X[:last_index_als+1], WIN_SIZE)) if last_index_als >= 0 else 0
    windowed_normal = len(windowing_fun(X[last_index_als+1:last_index_normal+1], WIN_SIZE)) if last_index_normal >= 0 else 0
    windowed_myopathie = len(windowing_fun(X[last_index_normal+1:], WIN_SIZE)) if last_index_myopathie >= 0 else 0
    
    return windowed_normal, windowed_als, windowed_myopathie


In [12]:
def last_index (Y):
    last_index_normal = len(Y) - 1 - Y[::-1].index(1)
    last_index_als = len(Y) - 1 - Y[::-1].index(0)
    last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)
    return last_index_normal, last_index_als, last_index_myopathie

In [13]:

# Sample file  name : N2001A03BB54
def generate_dataframe(data,files):
    #df = pd.DataFrame([], columns=["Class","Subject","Muscle","RecordID","Signal"])
    df = pd.DataFrame([], columns=['Class',"SubjectID","Muscle","RecordID","Signal"])
    windowed_data = [{"Signal": data[j],"Class":files[j][5],"SubjectID": files[j][6:8],"Muscle": files[j][8:10],"RecordID":files[j][10:12] } for j in range(0,len(data))]
    final_data = [windowed_data[i] for i in range(len(windowed_data))]
    df =df.concat(final_data) #, ignore_index=True
    return df

In [14]:
def NaN_values(signal_data_array):
    # Check if there are any NaN values in the data
    nan_indices = np.argwhere(np.isnan(signal_data_array))

    # Check if there are any empty values in the data
    empty_indices = np.argwhere(np.array([len(x) == 0 for x in signal_data_array]))
    np.set_printoptions(threshold=np.inf)

     #Print the indices of any NaN or empty values
    if len(nan_indices) > 0:
        print("Found NaN values ")
    else:
        print("none")
    if len(empty_indices) > 0:
        print("Found empty values ")
    else:
        print("none")
    nan_indices.shape

In [ ]:
# Get runing start time
start_time = time.time()

In [15]:
# Read data
database_dir = '/kaggle/input/emglab'

hea_files = [f for f in os.listdir(database_dir)]

# Initialize empty arrays to store the signal data and record names
signal_data_list = []
record_name_list = []
# Loop over each .hea file and read its signal data
for file in hea_files:
    if file.endswith(".hea"):
        record_name = os.path.splitext(file)[0] # Get the record name from the file name
        record_path = os.path.join(database_dir, record_name) # Get the full path to the record
        record = wfdb.rdrecord(record_path) # Read the record
        signal_data = np.array(record.p_signal) # Get the signal data as a numpy array
        signal_data_list.append(signal_data) # Append the signal data to the list
        record_name_list.append(record_name) # Append the record name to the list

# Convert the list of signal data arrays to a 3D numpy array
signal_data_array = np.stack(signal_data_list)

print(len(record_name_list))
print(signal_data_array.shape)

['N2001C03BB51', 'N2001C07BB51', 'N2001M02BB51', 'N2001C06BB65', 'N2001C07BB63', 'N2001C07BB76', 'N2001A03BB66', 'N2001C09BB51', 'N2001A04BB62', 'N2001C01BB61', 'N2001M03BB03', 'N2001M05BB60', 'N2001M07BB20', 'N2001M02BB61', 'N2001C06BB51', 'N2001A04BB60', 'N2001M03BB10', 'N2001A06BB11', 'N2001C03BB80', 'N2001C04BB73', 'N2001C09BB67', 'N2001C06BB57', 'N2001C04BB66', 'N2001M03BB08', 'N2001M07BB10', 'N2001A04BB66', 'N2001M02BB06', 'N2001A06BB14', 'N2001C06BB61', 'N2001C03BB64', 'N2001A06BB09', 'N2001A07BB02', 'N2001M03BB04', 'N2001C07BB57', 'N2001C09BB59', 'N2001A04BB51', 'N2001C04BB52', 'N2001C02BB76', 'N2001A06BB07', 'N2001C06BB73', 'N2001C01BB74', 'N2001M02BB59', 'N2001C04BB63', 'N2001C09BB57', 'N2001C02BB70', 'N2001C05BB54', 'N2001M04BB52', 'N2001C02BB65', 'N2001C06BB77', 'N2001A06BB18', 'N2001C07BB66', 'N2001C01BB68', 'N2001C08BB60', 'N2001C07BB59', 'N2001M04BB54', 'N2001A03BB54', 'N2001A01BB05', 'N2001A03BB51', 'N2001C08BB80', 'N2001A08BB03', 'N2001C01BB59', 'N2001C09BB78', 'N2001C

In [16]:
# Print data stats
f=getStat('BB','N2001M' , record_name_list)
print('number of myopathy signals',f)
f= getStat('BB','N2001A' , record_name_list)
print('number of ALS signals',f)
f=getStat('BB','N2001C' , record_name_list)
print('number of Control  signals',f)

number of signals taken from patients with myopathy from the biceps brachii muscle  107
number of signals taken from patients with als from the biceps brachii muscle  98
number of signals taken from normal people from the biceps brachii muscle  270


In [21]:
# Combine the labels and signals arrays into a list of tuples
data = list(zip(record_name_list, signal_data_array))

# Sort the list of tuples by the first element (the labels)
sorted_data = sorted(data, key=lambda x: x[0])

# Extract the sorted signals into a new array
signal_data_array = [x[1] for x in sorted_data]
record_name_list = [x[0] for x in sorted_data]

In [22]:
# Why  deleting 290:368 (78 control records) ?? needs explanation !!
signal_data_array = np.delete(signal_data_array,np.s_[290:368], axis=0)
record_name_list = np.delete(record_name_list,np.s_[290:368], axis=0)
print(np.shape(signal_data_array))
print(np.shape(record_name_list))

In [24]:
f=getStat('BB','N2001A' , record_name_list)
print('Number of ALS signals',f)
f=getStat('BB','N2001M' , record_name_list)
print('Number of myopathy signals',f)
f=getStat('BB','N2001C' , record_name_list)
print('number of Control signals',f)

number of signals taken from patients with ALS from the biceps brachii muscle  98
number of signals taken from patients with myopathy from the biceps brachii muscle  107
number of signals taken from normal people from the biceps brachii muscle  192


In [26]:
#Check for NaN values
NaN_values(signal_data_array)

Found NaN values 
none


In [27]:
df = generate_dataframe(signal_data_array,record_name_list)
df

In [29]:
def codeClasses(row):
    return {"C":1,"M":2, "A":0}[row["Class"]]

df["ClassID"] =df.apply(codeClasses, axis=1)
df
del signal_data_array,record_name_list

In [32]:
#  Data  stats: classes, subjects and records
for c in df['Class'].unique():
    print("",f"Class {c}:", sep="\n")
    print("Subjects Count")
    print(df[df['Class']==c]['SubjectID'].value_counts())


signal A:
pat count
03    20
04    20
06    18
08    15
01    12
07    10
05     3
Name: num_personne, dtype: int64

signal C:
pat count
01    30
02    30
03    30
04    30
05    30
06    30
07    12
Name: num_personne, dtype: int64

signal M:
pat count
02    25
07    22
03    16
05    16
06    16
04    12
Name: num_personne, dtype: int64


In [33]:
# Split data acording to suject specific strategy

ValSubjectID = "07"
TestSubjectID = "06"

print("Train")
df_train = df[~df["SubjectID"].isin([ValSubjectID,TestSubjectID])]
print(df_train["SubjectID"].value_counts())
print(df_train['Class'].value_counts())

print("Validation")
df_val = df[df["SubjectID"] == ValSubjectID]
df_val['Class'].value_counts()

print("Test")
df_test = df[df["SubjectID"] == TestSubjectID]
df_test['Class'].value_counts()

03    66
04    62
02    55
05    49
01    42
08    15
Name: num_personne, dtype: int64
C    150
A     70
M     69
Name: type_signal, dtype: int64


In [37]:
for signal in df_test['Class'].unique():
    print("",f"signal {signal}:", sep="\n")
    print("pat count")
    print(df_test[df_test['Class']==signal]['SubjectID'].value_counts())


signal A:
pat count
04    33
01    26
03    20
06    15
05    13
07    12
Name: num_personne, dtype: int64

signal M:
pat count
07    19
03    17
06    17
01    15
04    15
05    15
02    12
Name: num_personne, dtype: int64


In [39]:
X_train = df_train['signal'].tolist()
Y_train = df_train['ClassID'].tolist()
#print(Y_train)
print(np.asarray(X_train).shape)
print(np.asarray(Y_train).shape)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(289,)
(289,)


In [40]:
X_val = df_val['signal'].tolist()
Y_val = df_val['ClassID'].tolist()
#print(Y_val)
print(np.asarray(X_val).shape)
print(np.asarray(Y_val).shape)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(44,)
(44, 262124, 1)


In [41]:
X_test = df_test['signal'].tolist()
Y_test = df_test['ClassID'].tolist()
#print(Y_test)
print(np.asarray(Y_test).shape)
print(np.asarray(X_test).shape)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(32,)
(32, 262124, 1)


In [42]:
last_index_normal, last_index_als, last_index_myopathie = last_index (Y_train)
last_index_normal, last_index_als, last_index_myopathie

(219, 69, 288)

In [43]:
del df,df_train,df_test,df_val

In [44]:
#Windowing
windowed_normal_train,  windowed_als_train, windowed_myopathie_train = getClassesStat(Y_train, X_train)
windowed_normal_val, windowed_als_val, windowed_myopathie_val = getClassesStat(Y_val, X_val)
windowed_myopathie_test, windowed_als_test = getClassesStat(Y_test, X_test)

In [45]:
Y_train = [0]*(windowed_als_train)+[1]*(windowed_normal_train)+[2]*(windowed_myopathie_train)
print(np.asarray(Y_train).shape)
Y_val = [0]*(windowed_als_val)+[1]*(windowed_normal_val)+[2]*(windowed_myopathie_val)
print(np.asarray(Y_val).shape)
Y_test = [0]*(windowed_als_test)+[1]*(windowed_normal_val)+[2]*(windowed_myopathie_test)
print(np.asarray(Y_test).shape)

(7514,)
(1144,)
(832,)


In [46]:
print(np.asarray(X_train).shape)
print(np.asarray(X_val).shape)
print(np.asarray(X_test).shape)

(289, 262124, 1)
(44, 262124, 1)
(32, 262124, 1)


In [47]:
X_train = windowing_fun(X_train, WIN_SIZE)
X_val = windowing_fun(X_val, WIN_SIZE)
X_test = windowing_fun(X_test, WIN_SIZE)

print(np.shape(X_train))
print(np.shape(Y_train))

print(np.shape(X_val))
print(np.shape(Y_val))

print(np.shape(X_test))
print(np.shape(Y_test))

(832, 23437, 1)
(832,)
(1144, 23437, 1)
(1144,)
(7514, 23437, 1)
(7514,)


In [49]:
X_train,Y_train = remove_nan(X_train,Y_train)
X_val,Y_val = remove_nan(X_val,Y_val)
X_test,Y_test = remove_nan(X_test,Y_test)
print(np.shape(Y_test))
print(np.shape(X_test))

print(np.shape(X_val))
print(np.shape(Y_val))

print(np.shape(X_train))
print(np.shape(Y_train))

(826,)
(826, 23437, 1)
(1144, 23437, 1)
(1144,)
(7474, 23437, 1)
(7474,)


In [51]:
Y_train = list(Y_train)
last_index_normal, last_index_als, last_index_myopathie = last_index (Y_train)
last_index_normal, last_index_als, last_index_myopathie

(5689, 1791, 7473)

In [52]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_train)

# Check the new class distribution
print(Y_train_series.value_counts())

1    3898
0    1792
2    1784
dtype: int64


In [53]:
X_train_als = X_train[:last_index_als + 1]
X_train_normal = X_train[last_index_als + 1:last_index_normal + 1]
X_train_myopathy = X_train[last_index_normal + 1:]

In [54]:
num_samples_als, num_timesteps_als, num_channels_als = X_train_als.shape
X_train_als = X_train_als.reshape(num_samples_als, num_channels_als * num_timesteps_als)


num_samples_normal, num_timesteps_normal, num_channels_normal = X_train_normal.shape
X_train_normal = X_train_normal.reshape(num_samples_normal, num_channels_normal * num_timesteps_normal)


num_samples_myopathy, num_timesteps_myopathy, num_channels_myopathy = X_train_myopathy.shape
X_train_myopathy = X_train_myopathy.reshape(num_samples_myopathy, num_channels_myopathy * num_timesteps_myopathy)
X_train_normal.shape,X_train_als.shape,X_train_myopathy.shape

((3898, 23437), (1792, 23437), (1784, 23437))

In [55]:
normal_als = np.concatenate((X_train_normal,X_train_als))
normal_myopathy = np.concatenate((X_train_normal,X_train_myopathy))
Y_normal_als = [1] * len(X_train_normal) + [0] * len(X_train_als)
Y_normal_myopathy = [1] * len(X_train_normal) + [2] * len(X_train_myopathy)

# Apply SMOTE to the training set
smote = SMOTE()
normal_als, Y_normal_als = smote.fit_resample(normal_als, Y_normal_als)
# Apply SMOTE to the training set
smote = SMOTE()
normal_myopathy, Y_normal_myopathy = smote.fit_resample(normal_myopathy, Y_normal_myopathy)

In [56]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_normal_als)

# Check the new class distribution
print(Y_train_series.value_counts())

# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_normal_myopathy)

# Check the new class distribution
print(Y_train_series.value_counts())

1    3898
0    3898
dtype: int64
1    3898
2    3898
dtype: int64


In [57]:
def first_last_index(arr):
    first_index = -1
    last_index = -1
    for i in range(len(arr)):
        if arr[i] == 0:
            if first_index == -1:
                first_index = i
            last_index = i
    return first_index, last_index

In [58]:
first_index, last_index = first_last_index(Y_normal_als)

In [59]:
Y_train_als = Y_normal_als[first_index:last_index+1]
X_train_als = normal_als[first_index:last_index+1]

In [60]:
X_train = np.concatenate((X_train_als,normal_myopathy))
Y_train = np.concatenate((Y_train_als,Y_normal_myopathy))

In [61]:
#print(Y_train)

In [62]:
del X_train_als, normal_myopathy, Y_train_als, Y_normal_myopathy, Y_normal_als, normal_als

In [63]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_train)

# Check the new class distribution
print(Y_train_series.value_counts())

0    3898
1    3898
2    3898
dtype: int64


In [64]:
X_train = gen_tensors_list(X_train,(WIN_SIZE,1))
X_val = gen_tensors_list(X_val,(WIN_SIZE,1))
X_test = gen_tensors_list(X_test,(WIN_SIZE,1))

In [65]:
print(np.shape(X_test))
print(np.shape(X_train))
print(np.shape(X_val))

print(np.shape(Y_test))
print(np.shape(Y_train))
print(np.shape(Y_val))

(826, 23437, 1)
(11694, 23437, 1)
(1144, 23437, 1)
(826,)
(11694,)
(1144,)


In [ ]:
print("--- Data preparation time: %s seconds ---" % (time.time() - start_time))

In [66]:
model = Sequential([
    
    #Conv1D(#of filters, filters size, activation getStat)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(32,  kernel_size=13, padding='valid', dilation_rate=1,activation = 'relu', input_shape = (WIN_SIZE,1),kernel_initializer='he_normal'), 
    
    BatchNormalization(),
    Dropout(0.2),
    #LAYER2
    Conv1D(64, kernel_size=11, padding='valid' , dilation_rate=2, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001)),
    
    BatchNormalization(),
 
    Dropout(0.3),
    #LAYER3
    Conv1D(128, kernel_size=9, padding='valid', dilation_rate=3, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.001)),
    
    BatchNormalization(),
    
    Dropout(0.4),
    
    Conv1D(256, kernel_size=7, padding='valid', dilation_rate=4, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    
    BatchNormalization(),
    
    #GlobalMaxPooling1D layer
    GlobalMaxPooling1D(data_format = 'channels_last'),
    
   # Flatten(),
    
    #FC Layers:
    
    #LAYER4
    #Dense : fully connected  Dense(#neurons, activation getStat )
    
    Dense(128, activation = 'relu'),
    
    BatchNormalization(),
    
    Dropout(0.3),
    
    #LAYER5
    Dense(3, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 23425, 32)         448       
                                                                 
 batch_normalization (BatchN  (None, 23425, 32)        128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 23425, 32)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 23405, 64)         22592     
                                                                 
 batch_normalization_1 (Batc  (None, 23405, 64)        256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 23405, 64)         0

In [ ]:
# Set the initial learning rate
initial_lr = 0.001

# Create the Adam optimizer
opt = Adam(learning_rate = initial_lr, beta_1=0.9, beta_2=0.999, amsgrad=False, epsilon=None)

# Create the ReduceLROnPlateau callback to decrease the learning rate by a factor of 10
# when the validation loss doesn't improve for 5 consecutive epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_lr=1e-10)

# Create the EarlyStopping callback to stop training if the validation loss doesn't improve
# for 20 consecutive epochs
early_stop = EarlyStopping(monitor='val_loss', patience=25, verbose=1)

# Compile the model
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

Epoch 1/100
 95/731 [==>...........................] - ETA: 2:08:31 - loss: 5.9113 - sparse_categorical_accuracy: 0.5230

In [ ]:
tr_start=time.time()
# Train the model with early stopping
history = model.fit( 
    x = np.asarray(X_train),
    y = np.asarray(Y_train),
    #validation_split=0.2,
    validation_data = (np.asarray(X_val), np.asarray(Y_val)),
    shuffle=True,
    epochs=100,
    batch_size=16,
    callbacks=[reduce_lr, early_stop]

)
print("--- Training time: %s seconds ---" % (time.time() - tr_start))


In [ ]:
#Save trained model
model.save("Model_vl_ts.h5")

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
  
print(history.history.keys())

In [ ]:
#Prediction
#scores = model.evaluate(np.asarray(X_test),np.asarray (Y_test))
y_pred = model.predict(np.asarray(X_test)) 

In [ ]:
#y_predicted_labels = [{0:"Neuropathie",1:"Normal",2:"Myopathy"}[i] for i in np.argmax(y_pred,axis=1)]
#y_true_labels = [{0:"Neuropathie",1:"Normal",2:"Myopathy"}[i] for i in Y_test]

In [ ]:
#Confusion Matrix
con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=np.argmax(y_pred,axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = ["Neuropathie","Normal","Myopathie"], 
                     columns = ["Neuropathie","Normal","Myopathie"])

figure = plt.figure(figsize=(5,5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
#Classification Report
predictions=np.argmax(y_pred,axis=1)
# y_true is the ground truth labels
# y_pred is the predicted labels
print(classification_report(Y_test, predictions))

In [ ]:
print("--- Total running time: %s seconds ---" % (time.time() - start_time))